# 4-1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
from category_encoders import *
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from scipy import stats
from sklearn.linear_model import LogisticRegression

In [ ]:
df = pd.read_csv("Week 4 Data/iris.csv")

In [ ]:
df.head()

In [ ]:
#statistics
df.describe()

In [ ]:
#null check
df.isnull().any()

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data=df)

In [ ]:
#Removing Outliers based on IQR
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
#Series comparison with each column of df 
#(df[df[col]<x] 이건 df의 series의 값을 scalar와 비교, df<Series 이건 df의 series를 다른 series와 비교)
df_refined = df[~((df<(Q1-1.5*IQR))|(df>(Q3+1.5*IQR))).any(axis=1)]

In [ ]:
#correlation check
corr = df_refined.corr()
corr.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
#encode class col
enc = BinaryEncoder(cols=['class']).fit(df_refined)
enc_df = enc.transform(df_refined)

In [ ]:
enc_df.head()

In [ ]:
#df to lists
X = enc_df.loc[:,enc_df.columns != 'petal width'].values
y = enc_df.loc[:,enc_df.columns == 'petal width'].values.ravel()

In [ ]:
#dummy trap 피하기
X = X[:,1:]

In [ ]:
#train_test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [ ]:
#model instantiate
regressor = LinearRegression()
regressor.fit(X_train,y_train)

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
regressor.score(X_test,y_test)

In [ ]:
plt.figure(figsize=(20, 6))
list_data = [y_test, y_pred]
ax = sns.lineplot(data=list_data)

# 4-2

In [ ]:
df = pd.read_csv('Week 4 Data/cell_samples_string.csv')
df.drop(columns=['ID'],inplace=True)
df.head()

In [ ]:
#Statistics table
df.describe()

In [ ]:
#boxplot
plt.figure(figsize=(10, 10))
sns.boxplot(data=df)

In [ ]:
#Remove outlier based on IQR
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3-Q1
print(IQR)

In [ ]:
df_refined = df[~((df<(Q1-1.5*IQR))|(df>(Q3+1.5*IQR))).any(axis=1)]

In [ ]:
df_refined.describe()

In [ ]:
#Removing Mit since 0 variance
df_refined.drop(columns=['Mit'],inplace=True)

In [ ]:
df_refined

In [ ]:
#correlation check
corr = df_refined.corr()
corr.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
#다중공선성을 띄는 UnifSize와 UnifShape중 어떤걸 떨궈야할까?
print(corr.UnifSize.sum())
print(corr.UnifShape.sum())

In [ ]:
#다른변수에 더 영향을 미치는 UnifSize를 drop하겠음.
df_refined.drop(columns=['UnifSize'],inplace=True)

In [ ]:
#LDA 차원축소를 위해 X,y로 데이터를 나눔.
X = df_refined.iloc[:,:-1].values
y = df_refined.iloc[:,-1].values

In [ ]:
clf = LinearDiscriminantAnalysis()
clf.fit_transform(X,y)

In [ ]:
clf.fit_transform(X,y).ravel()

In [ ]:
plt.figure(figsize=(10, 10))
ax = sns.scatterplot(x = y,y=clf.fit_transform(X,y).ravel(),hue = y)

# 4-3

In [ ]:
#import data
df = pd.read_csv('Week 4 Data/vehicle.csv').iloc[:,1:]
df.head()

In [ ]:
#data statistics
df.describe()

In [ ]:
#null check
df.isnull().any()

In [ ]:
df.skew()

In [ ]:
#Removing Col 4,5 for std <0.1, Col 7 for skew <0.1
df.drop(columns=['Col4','Col5','Col7'],inplace=True)

In [ ]:
#boxplot
plt.figure(figsize=(10, 10))
sns.boxplot(data=df)

In [ ]:
#removing outliers based on IQR
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3-Q1
df_refined = df[~((df < (Q1 - 1.5 * IQR))| (df > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data=df_refined)

In [ ]:
#correlation check
corr = df_refined.corr()
corr.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
#너무엉망이라 걍 이번 실습에서는 그대로 다 집어넣고 가겠음.
#X,y로 나누고 partitioning
X = df_refined.iloc[:,:-1].values
y = df_refined.iloc[:,-1].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
#gridsearch로 best parameter optimization 실시
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

svc_param_selection(X_train,y_train,10)

In [ ]:
clf = svm.SVC(C=10,gamma=1)
clf.fit(X_train,y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
cm=confusion_matrix(y_test,y_pred)
cm

In [ ]:
#Accuracy of optimized rbf hyperparameters
clf.score(X_test,y_test)

# 4-4

In [ ]:
#import data with dropping unnecessary columns
df = pd.read_csv('Week 4 Data/Churn123.txt', sep=",", header=0).iloc[:,1:-1]
df.head()

In [ ]:
#Added new column
df['Leaver'] = df['CHURNED'].apply(lambda x: 1 if x == "'Vol'" else 0)

In [ ]:
#Data Statistics
df.describe()

In [ ]:
#normalize using z-score
df = df.apply(lambda x: stats.zscore(x) if x.name == 'LONGDIST' or x.name == 'International' or x.name == 'LOCAL' or x.name == 'DROPPED' or x.name == 'AGE' or x.name == 'CHILDREN' or x.name == 'Est_Income' else x)

In [ ]:
df.describe()

In [ ]:
#box-plot
plt.figure(figsize=(10,10))
sns.boxplot(data=df)

In [ ]:
#outlier drop
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
df_refined = df[~((df < Q1-1.5*IQR)|(df > Q3+1.5*IQR)).any(axis=1)]

In [ ]:
df_refined.describe()

In [ ]:
enc = BinaryEncoder(cols=['PAY_MTHD', 'LocalBillType','LongDistanceBillType','SEX','STATUS','Car_Owner']).fit(df_refined)
enc_df = enc.transform(df_refined)

In [ ]:
enc_df.columns

In [ ]:
#X,y Split
X = enc_df.iloc[:,:-2].values
y = enc_df.iloc[:,-1].values

In [ ]:
#train_test_sp,it
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [ ]:
clf = LogisticRegression(solver='sag',max_iter=2000)
clf.fit(X_train,y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
clf.score(X_test,y_test)